In [1]:
import pandas as pd
import numpy as np
import os
import healpy as hp
from astroquery.vizier import Vizier
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
import math

In [2]:
'''Set of functions for getting a array of star list from [radsec]-area around the object, 
transformation the information to common view according [info] and writting into DataFrame.'''



def RenameColumns(table, catalogue):
    global info
    
    old_columns = info.drop(['link', 'pecularity'], axis = 1)
    new_columns = info.drop(['link', 'pecularity'], axis = 1)
    new_columns = new_columns.rename(columns = {'phot_g_mean_mag': 'mag'})
        
    old_columns = old_columns.loc[catalogue].tolist()
    new_columns = new_columns.columns.tolist()
            
    for i in range(len(new_columns)):
        table = table.rename(columns= {old_columns[i]: new_columns[i]})
    return table

def errors_2MASS(row):
    a = row['errMaj']
    b = row['errMin']
    angle = row['errPA']
    row['errMaj'] = abs(a * b) / (math.sqrt(
        (a ** 2) * (math.cos(math.radians(angle)) ** 2) + 
        (b ** 2) * (math.sin(math.radians(angle)) ** 2)))
    row['errMin'] = abs(a * b) / (math.sqrt(
        (a ** 2) * (math.sin(math.radians(angle)) ** 2) + 
        (b ** 2) * (math.cos(math.radians(angle)) ** 2)))
    return row

def epoch_UCAC4(row):
    a = row['EpRA']
    b = row['EpDE']
    row['EpRA'] = (a + b)/2
    return row

def CatDataFrame(star, catalogue):
    global info
    global radsec
    try:
        link = info.loc[catalogue]['link']
        viz = Vizier(columns=['all'])
        table = viz.query_region(coord.SkyCoord(
            ra=star.ra, dec=star.dec, unit=(u.deg, u.deg), frame='icrs'),
                                 radius=radsec*u.arcsec, catalog=link)[0]
        
        df = pd.DataFrame.from_records(table.as_array())
        
        pecularities = info.pecularity[catalogue]
        pecs = pecularities.split('|')
            
        if (not pecs[0] == 'mas'):
                
            column1 = info ['ra_error'][catalogue]
            column2 = info ['dec_error'][catalogue]
                    
            df[column1] = df[column1].apply(lambda x: x*1000)
            df[column2] = df[column2].apply(lambda x: x*1000)
                    
            if len(pecs[0].split(':')) == 2:
                df[['errMaj', 'errMin', 'errPA']] = \
                df[['errMaj', 'errMin', 'errPA']].apply(errors_2MASS, axis = 1)
                        
                
        if (not pecs[1] == 'jy'):
            ps = pecs[1].split(':')
                    
            if len(ps) == 2:
                column = info ['ref_epoch'][catalogue]
                
                df[column] = \
                df[column].apply(lambda x: Time((x+ int(ps[0])), format=ps[1]).jyear)
                
            else:
                df[['EpRA','EpDE']] = \
                df[['EpRA','EpDE']].apply(epoch_UCAC4, axis = 1)

        cols = info.loc[catalogue].tolist()[:-2]
        df = df[['_r'] + cols]
        df = RenameColumns(df, catalogue)
        df['delta_mag'] = df.mag
        df.delta_mag = df.delta_mag.apply(lambda x: abs(x-star.mag))
        first_col = df.pop('delta_mag')
        df.insert(0, 'delta_mag', first_col)
        df.delta_mag = round(df.delta_mag, 2)
        df._r = round(df._r, 4)
        df.delta_mag = round(df.delta_mag, 2)
        df.ref_epoch = round(df.ref_epoch, 1)
        
        print(catalogue)# + ': success')
        return [df, True]
    
    except Exception as e:
        print(catalogue + '\t' + ' ERROR:', e)
        return ['NaN', False]

In [3]:
def best_match(df):
    global radsec
    global min_delta_mag
    number = 0
    if len(df) > 1:
        no_mag = True
        minimal = min_delta_mag
        for i in range(len(df)):
            s = df.iloc[i]
            if s.delta_mag < minimal:
                no_mag = False
                minimal = s.delta_mag
                number = i
        if no_mag:
            distance = radsec
            for i in range(len(df)):
                s = df.iloc[i]
                if s._r < distance:
                    distance = s._r
                    number = i
    s = df.drop('mag', axis = 1).iloc[number].apply(str)
    list_s = s.tolist()
    return '|'.join(list_s)

In [4]:
def tangFromRADE(ra, de, RA, DE):
    ksi = math.cos(de)*math.sin(ra-RA)/(math.sin(de)*math.sin(DE)+math.cos(de)*math.cos(DE)*math.cos(ra-RA))
    eta = (math.sin(de)*math.cos(DE)-math.cos(de)*math.sin(DE)*math.cos(ra-RA))/(math.sin(de)*math.sin(DE)+math.cos(de)*math.cos(DE)*math.cos(ra-RA))
    return ksi,eta


def RADecFromTang(ksi, eta, RA, Dec):
    secRho = math.sqrt(1+ksi*ksi+eta*eta)
    t11 = -math.sin(RA)
    t12 = math.cos(RA)
    t13 = 0
    t21 = -math.cos(RA)*math.sin(Dec)
    t22 = -math.sin(RA)*math.sin(Dec)
    t23 = math.cos(Dec)
    t31 = math.cos(RA)*math.cos(Dec)
    t32 = math.sin(RA)*math.cos(Dec)
    t33 = math.sin(Dec)
    x = (ksi*t11+eta*t21+t31)/secRho
    y = (ksi*t12+eta*t22+t32)/secRho
    z = (ksi*t13+eta*t23+t33)/secRho
    ra = math.atan2(y,x)
    dec = math.atan2(z,math.sqrt(x*x+y*y))
    if(ra<0):
        ra+=2*math.pi
    return ra,dec

def calibration(x,y,ksi,eta,n,Nmin,rmax):
    Q = int((n+1)*(n+2)/2)
    C = np.ones((np.size(x), Q))
    q = 0
    for i in range(n + 1):
        for j in range(n + 1):
            if (i + j <= n):
                C[:, q] = (x ** i) * (y ** j)
                q += 1
    We = np.diag(np.ones(np.size(x)))
    flag = 0
    it = 0
    while (flag == 0):
        Zx = np.dot(np.linalg.inv(np.dot(np.dot(np.transpose(C), We), C)),\
                    np.dot(np.dot(np.transpose(C), We), ksi))
        Zy = np.dot(np.linalg.inv(np.dot(np.dot(np.transpose(C), We), C)),\
                    np.dot(np.dot(np.transpose(C), We), eta))
        rx = np.dot(We, ksi - np.dot(C, Zx))
        ry = np.dot(We, eta - np.dot(C, Zy))
        r = np.sqrt(rx ** 2 + ry ** 2)
        kmax = np.argmax(r)
        flag = 1
        if (np.size(ksi) - it <= Nmin):
            break
        if (r[kmax] > rmax):
            We[kmax, kmax] = 0
            flag = 0
            it += 1
    uwex = np.dot(np.dot(np.transpose(rx), We), rx) / (np.size(x) - it - Q)
    uwey = np.dot(np.dot(np.transpose(ry), We), ry) / (np.size(y) - it - Q)
    return Zx,Zy,math.sqrt(uwex),math.sqrt(uwey),np.size(x)-it


def transform(x,y,Z):
    Q = np.size(Z)
    n = int(np.max(np.roots(np.array([1,3,-2*(Q-1)]))))
    C = np.ones((np.size(x), Q))
    q = 0
    for i in range(n+1):
        for j in range(n+1):
            if (i + j <= n):
                C[:, q] = (x ** i) * (y ** j)
                q += 1
    return np.dot(C,Z)


def sqrerror(e0, e, tg, tc):
    e2 = e / (tg - tc)
    e2 = e2**2 + e0**2
    #e - error from calibration, e0 - pm error from gaia, tg - gaia epoch, tc - cat epoch
    return e2

In [5]:
'''Global variables'''
info = pd.read_csv('cat_info.csv').fillna('NaN')
radsec = 30
min_delta_mag = 10
info

,ra,dec,ra_error,dec_error,ref_epoch,phot_g_mean_mag,link,pecularity
CMC15,RA_ICRS,DE_ICRS,e_RA_ICRS,e_DE_ICRS,MJD-51263,r_mag,I/327/cmc15,arcsec|51263:mjd
Pan-STARRS DR1,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,_tab1_10,gmag,II/349/ps1,arcsec|0:mjd
2MASS All-Sky,RAJ2000,DEJ2000,errMaj,errMin,_tab1_36,Jmag,II/246/out,arcsec:errPA|0:jd
SDSS DR12,RA_ICRS,DE_ICRS,e_RA_ICRS,e_RA_ICRS,ObsDate,gmag,V/147/sdss12,arcsec|jy
URAT1,RAJ2000,DEJ2000,sigs,sigm,Epoch,gmag,I/329/urat1,mas|jy
UCAC4,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,EpRA,gmag,I/322A/out,mas|EpDE


In [6]:
#new_columns = info.drop(['link', 'pecularity'], axis = 1)#.columns
#new_columns = new_columns.rename(columns = {'phot_g_mean_mag': 'mag'}).columns.tolist()
#new_columns

In [7]:
directory = 'catalog_positions'
os.listdir(directory)

['Pan-STARRS DR1.csv',
 'URAT1.csv',
 '2MASS All-Sky.csv',
 'SDSS DR12.csv',
 'CMC15.csv',
 'UCAC4.csv']

In [8]:
for file in os.listdir(directory):
    filepath = os.path.join(directory, file)
    df = pd.read_csv(filepath, index_col = 0)

In [9]:
display(file.split('.')[0], df.head())

'UCAC4'

,gaia_ra,gaia_dec,gaia_ra_error,gaia_dec_error,pmra,pmdec,pmra_error,pmdec_error,gaia_epoch,cat_ra,cat_dec,cat_epoch,delta_mag,dist_sec
source_id,,,,,,,,,,,,,,
4892429171367335040,65.400190,-27.046093,0.034936,0.045501,-12.468865,20.545892,0.081924,0.094741,2015.5,65.400243,-27.046217,1977.0,2.22,0.460
6027693584407085056,254.393003,-31.375706,1.491057,0.933969,-12.919468,8.647653,2.627126,1.571694,2015.5,254.384213,-31.377857,1985.9,NaN,28.134
6027685097545659136,254.298860,-31.520545,1.537464,1.140934,-5.669891,8.700257,2.801365,1.819132,2015.5,254.290053,-31.519265,1997.2,5.79,27.441
6027685544222718208,254.071639,-31.628937,2.435719,0.998516,-2.753096,3.776416,2.540000,1.694394,2015.5,254.076908,-31.635925,1996.2,4.33,29.894
6027681558489999232,254.175604,-31.674378,1.565271,1.089150,-13.301846,7.345171,3.101950,1.795421,2015.5,254.172660,-31.680529,1996.4,NaN,23.936


In [12]:
j = input('драгоценности: ')
s = input('камни: ')
count = 0
for e in s:
    if e in j:
        count +=1
print (count)

драгоценности: jrehglkjarehlgjk
камни: jhgjhgk
7


In [20]:
n = 4
vector = []
for i in range(n):
    a = str(input('число меньше 10000: '))
    vector.append(a)
count = 0
for a in vector:
    if '1'*(count+1) in a:
        count +=1
print(count)
        


число меньше 10000: 1
число меньше 10000: 1
число меньше 10000: 4
число меньше 10000: 21
1


###### 

1
14235
11342
21135
q123454u685
225366352
2
